# TODO: Zklend

In [ ]:
from typing import Dict
import collections
import decimal

import IPython.display
import pandas

import constants
import db

## Load and prepare events

In [ ]:
# Establish the connection.
connection = db.establish_connection()

In [ ]:
# Load all Zklend events.
zklend_events = pandas.read_sql(
    sql =  """
        SELECT
            *
        FROM
            starkscan_events
        WHERE
            from_address='{}';
    """.format(constants.Protocol.ZKLEND.value),
    con = connection,
)

In [ ]:
# Close the connection.
connection.close()

In [ ]:
# TODO: move to constants?
RELEVANT_EVENTS = {
    'Deposit',
    'Withdrawal',
    'CollateralEnabled',
    'CollateralDisabled',
    'Borrowing',
    'Repayment',
    'Liquidation',
    'AccumulatorsSync',
    'InterestRatesSync',
}

# Remove redundant data, sort remaining data and set index.
zklend_events.drop(
    zklend_events[~zklend_events['key_name'].isin(RELEVANT_EVENTS)].index,
    inplace = True,
)
zklend_events.sort_values('block_number', inplace = True)
zklend_events.set_index('id', inplace = True)

# Process events

In [ ]:
class AccumulatorState:
    '''
    TODO
    '''

    def __init__(self) -> None:
        self.lending_accumulator: decimal.Decimal = 1e27
        self.debt_accumulator: decimal.Decimal = 1e27

    def accumulators_sync(self, lending_accumulator: int, debt_accumulator: int):
        self.lending_accumulator = decimal.Decimal(lending_accumulator) / decimal.Decimal('1e27')
        self.debt_accumulator = decimal.Decimal(debt_accumulator) / decimal.Decimal('1e27')


class InterestRatesState:
    '''
    TODO
    '''

    def __init__(self) -> None:
        self.lending_rate: decimal.Decimal = 1e27
        self.borrowing_rate: decimal.Decimal = 1e27

    def interest_rates_sync(self, lending_rate: int, borrowing_rate: int):
        self.lending_rate = decimal.Decimal(lending_rate) / decimal.Decimal('1e27')
        self.borrowing_rate = decimal.Decimal(borrowing_rate) / decimal.Decimal('1e27')


class UserTokenState:
    '''
    TODO

    We are making a simplifying assumption that when collateral is enabled, all
    deposits of the given token are considered as collateral.
    '''

    def __init__(self, token: str) -> None:
        self.token: str = token
        self.deposit: int = 0
        self.collateral_enabled: bool = False
        self.borrowings: int = 0


class UserState:
    '''
    TODO
    '''

    def __init__(self) -> None:
        self.token_states: Dict[str, UserTokenState] = {
            'ETH': UserTokenState('ETH'),
            'wBTC': UserTokenState('wBTC'),
            'USDC': UserTokenState('USDC'),
            'DAI': UserTokenState('DAI'),
            'USDT': UserTokenState('USDT'),
        }
        self.health_factor: float = 1.0  # TODO: is this a good default value??

    def deposit(self, token: str, face_amount: int):
        self.token_states[token].deposit += face_amount

    def withdrawal(self, token: str, face_amount: int):
        self.token_states[token].deposit -= face_amount

    def collateral_enabled(self, token: str):
        self.token_states[token].collateral_enabled = True

    def collateral_disabled(self, token: str):
        self.token_states[token].collateral_enabled = False

    def borrowing(self, token: str, raw_amount: int, face_amount: int):
        self.token_states[token].borrowings += raw_amount

    def repayment(self, token: str, raw_amount: int, face_amount: int):
        self.token_states[token].borrowings -= raw_amount

    def liquidation(
        self,
        debt_token: str,
        debt_raw_amount: int,
        debt_face_amount: int,
        collateral_token: int,
        collateral_amount: int,
    ):
        self.token_states[debt_token].borrowings -= debt_raw_amount
        self.token_states[collateral_token].deposit -= collateral_amount


class State:
    '''
    TODO
    '''

    EVENTS_FUNCTIONS_MAPPING: Dict[str, str] = {
        'Deposit': 'process_deposit_event',
        'Withdrawal': 'process_withdrawal_event',
        'CollateralEnabled': 'process_collateral_enabled_event',
        'CollateralDisabled': 'process_collateral_disabled_event',
        'Borrowing': 'process_borrowing_event',
        'Repayment': 'process_repayment_event',
        'Liquidation': 'process_liquidation_event',
        'AccumulatorsSync': 'process_accumulators_sync_event',
        'InterestRatesSync': 'process_interest_rates_sync_event',
    }

    def __init__(self) -> None:
        self.user_states: collections.defaultdict = collections.defaultdict(UserState)
        self.accumulator_states: Dict[str, AccumulatorState] = {
            'ETH': AccumulatorState(),
            'wBTC': AccumulatorState(),
            'USDC': AccumulatorState(),
            'DAI': AccumulatorState(),
            'USDT': AccumulatorState(),
        }
        self.interest_rates_states: Dict[str, InterestRatesState] = {
            'ETH': InterestRatesState(),
            'wBTC': InterestRatesState(),
            'USDC': InterestRatesState(),
            'DAI': InterestRatesState(),
            'USDT': InterestRatesState(),
        }

    def process_event(self, event: pandas.Series) -> None:
        name = event['key_name']
        getattr(self, self.EVENTS_FUNCTIONS_MAPPING[name])(event = event)

    def process_deposit_event(self, event: pandas.Series) -> None:
        # The order of the arguments is: `user`, `token`, `face_amount`.
        user = event['data'][0]
        token = constants.get_symbol(event['data'][1])
        # TODO: divide by something or store like this?
        face_amount = int(event['data'][2], base = 16)
        # TODO: sanity checks/asserts?
        # TODO
        if user == '0x30b399e06903676ada3eccd5522e0cca4c4ad0101468c0ac407a56aa1a0ed3c' and token == 'DAI':
            print(
                'de',
                token,
#                 self.accumulator_states[token].lending_accumulator,
                self.interest_rates_states[token].lending_rate,
                face_amount,
            )
        # TODO: use raw_amount?
        # raw_amount = face_amount / self.accumulator_states[token].lending_accumulator
        self.user_states[user].deposit(token = token, face_amount = face_amount)

    def process_withdrawal_event(self, event: pandas.Series) -> None:
        # The order of the arguments is: `user`, `token`, `face_amount`.
        user = event['data'][0]
        token = constants.get_symbol(event['data'][1])
        face_amount = int(event['data'][2], base = 16)
        # TODO
        if user == '0x30b399e06903676ada3eccd5522e0cca4c4ad0101468c0ac407a56aa1a0ed3c' and token == 'DAI':
            print(
                'we',
                token,
#                 self.accumulator_states[token].lending_accumulator,
                self.interest_rates_states[token].lending_rate,
                face_amount,
            )
        # TODO: use raw_amount?
        # raw_amount = face_amount / self.accumulator_states[token].lending_accumulator
        self.user_states[user].withdrawal(token = token, face_amount = face_amount)

    def process_collateral_enabled_event(self, event: pandas.Series) -> None:
        # The order of the arguments is: `user`, `token`.
        user = event['data'][0]
        token = constants.get_symbol(event['data'][1])
        self.user_states[user].collateral_enabled(token = token)

    def process_collateral_disabled_event(self, event: pandas.Series) -> None:
        # The order of the arguments is: `user`, `token`.
        user = event['data'][0]
        token = constants.get_symbol(event['data'][1])
        self.user_states[user].collateral_disabled(token = token)

    def process_borrowing_event(self, event: pandas.Series) -> None:
        # The order of the arguments is: `user`, `token`, `raw_amount`, `face_amount`.
        user = event['data'][0]
        token = constants.get_symbol(event['data'][1])
        raw_amount = int(event['data'][2], base = 16)
        face_amount = int(event['data'][3], base = 16)  # TODO: relevant?
        self.user_states[user].borrowing(
            token = token,
            raw_amount = raw_amount,
            face_amount = face_amount,
        )

    def process_repayment_event(self, event: pandas.Series) -> None:
        # The order of the arguments is: `repayer`, `beneficiary`, `token`, `raw_amount`,
        # `face_amount`.
        repayer = event['data'][0]  # TODO: relevant?
        beneficiary = event['data'][1]
        token = constants.get_symbol(event['data'][2])
        raw_amount = int(event['data'][3], base = 16)
        face_amount = int(event['data'][4], base = 16)  # TODO: relevant?
        self.user_states[beneficiary].repayment(
            token = token,
            raw_amount = raw_amount,
            face_amount = face_amount,
        )

    def process_liquidation_event(self, event: pandas.Series) -> None:
        # The order of the arguments is: `liquidator`, `user`, `debt_token`, `debt_raw_amount`,
        # `debt_face_amount`, `collateral_token`, `collateral_amount`.
        liquidator = event['data'][0]  # TODO: relevant?
        user = event['data'][1]
        debt_token = constants.get_symbol(event['data'][2])
        debt_raw_amount = int(event['data'][3], base = 16)
        debt_face_amount = int(event['data'][4], base = 16)  # TODO: relevant?
        collateral_token = constants.get_symbol(event['data'][5])
        collateral_amount = int(event['data'][6], base = 16)
        # TODO
        if user == '0x30b399e06903676ada3eccd5522e0cca4c4ad0101468c0ac407a56aa1a0ed3c':
            print(
                'le',
                token,
#                 self.accumulator_states[token].lending_accumulator,
                self.interest_rates_states[token].lending_rate,
                collateral_amount,
            )
        self.user_states[user].liquidation(
            debt_token = debt_token,
            debt_raw_amount = debt_raw_amount,
            debt_face_amount = debt_face_amount,
            collateral_token = collateral_token,
            collateral_amount = collateral_amount,
        )

    def process_accumulators_sync_event(self, event: pandas.Series) -> None:
        # The order of the arguments is: `token`, `lending_accumulator`, `debt_accumulator`.
        token = constants.get_symbol(event['data'][0])
        lending_accumulator = int(event['data'][1], base = 16)
        debt_accumulator = int(event['data'][2], base = 16)
        self.accumulator_states[token].accumulators_sync(
            lending_accumulator = lending_accumulator,
            debt_accumulator = debt_accumulator,
        )

    def process_interest_rates_sync_event(self, event: pandas.Series) -> None:
        # The order of the arguments is: `token`, `lending_rate`, `borrowing_rate`.
        token = constants.get_symbol(event['data'][0])
        lending_rate = int(event['data'][1], base = 16)
        borrowing_rate = int(event['data'][2], base = 16)
        self.interest_rates_states[token].interest_rates_sync(
            lending_rate = lending_rate,
            borrowing_rate = borrowing_rate,
        )

In [ ]:
# Iterate over ordered events to obtain the final state of each user.
state = State()
for _, event in zklend_events.iterrows():
    state.process_event(event = event)

# TODO: Debugging negative deposits below

In [ ]:
# TODO: lending_rate
print(
    (900000000000000000000)
    + (80000000000000000000)
    + (502493389030314091303)
    - (1443577111469088876324)
    - (43000000000000000000)
)
print(
    (900000000000000000000 / 1)
    + (80000000000000000000 / 1.697027795157915701767592930)
    + (502493389030314091303 / 1.027844802691570804953811668)
    - (1443577111469088876324 / 1.027589274106706730393339514)
    - (43000000000000000000 / 1.079295108773576593430121832)
)
print(
    (900000000000000000000 * 1)
    + (80000000000000000000 * 1.697027795157915701767592930)
    + (502493389030314091303 * 1.027844802691570804953811668)
    - (1443577111469088876324 * 1.027589274106706730393339514)
    - (43000000000000000000 * 1.079295108773576593430121832)
)

In [ ]:
# TODO: lending_accumulator
print(
    (900000000000000000000)
    + (80000000000000000000)
    + (502493389030314091303)
    - (1443577111469088876324)
    - (43000000000000000000)
)
print(
    (900000000000000000000 / 1.000000000000000000000000000)
    + (80000000000000000000 / 1.000341640805821649966147094)
    + (502493389030314091303 / 1.006022702795017542366777716)
    - (1443577111469088876324 / 1.007697118768125486105104087)
    - (43000000000000000000 / 1.007707517677641512709962394)
)
print(
    (900000000000000000000 * 1.000000000000000000000000000)
    + (80000000000000000000 * 1.000341640805821649966147094)
    + (502493389030314091303 * 1.006022702795017542366777716)
    - (1443577111469088876324 * 1.007697118768125486105104087)
    - (43000000000000000000 * 1.007707517677641512709962394)
)

In [ ]:
# TODO: negative deposits?
for attr in ['deposit', 'borrowings']:
    for token in ['ETH', 'wBTC', 'USDC', 'DAI', 'USDT']:
        values = [getattr(user_state.token_states[token], attr) for user, user_state in state.user_states.items()]
        print(attr, token, pandas.Series(values).describe())

In [ ]:
# TODO: show negative deposits
[
    (user, user_state.token_states['DAI'].deposit)
    for user, user_state
    in state.user_states.items()
    if user_state.token_states['DAI'].deposit < 0
]